In [1]:
import requests

In the next block, you just need to fill in some API URL. For example, put "http://chroniclingamerica.loc.gov/search/pages/results/?state=Arizona&format=json" in there.

In [2]:
r = requests.get("http://chroniclingamerica.loc.gov/search/pages/results/?state=Arizona&format=json").json()

Now, we have that page (go ahead, you can click on it to look at it and see what it looks like) availble to us in this little script. We can already do some cool stuff:

 - how many total items are there in this search?

In [3]:
r['totalItems']

311217

Easy peasy! Now let's look at some of the items:

In [10]:
page_url = r['items'][0]['url']
print(page_url)

http://chroniclingamerica.loc.gov/lccn/sn84020558/1906-06-10/ed-1/seq-2.json


In [6]:
r['items'][0]['ocr_eng']

'TilU AIUZOXA JIU\'U1I1jICA: SUNDA.Y SlOIiMKU, JLTKK lO, 1\'JOO\nTHE ARIZONA REPUBLICAN\nPublished Every Day in the Year bj\nTHE\nArizona Publishing Co.\nGEO. W. VICKERS. Pres. and Cen. Mg.\nExclusive Morning Associated Press\nP\'spatches. . ,\nThe only Perfecting Prss In Arizona\nThe only battery of Linotypes In\nArizona.\nPublication office: 35-38 East Adams\nstreet. Telephone Main 47.\nEntered at the postoffiee at Phoenix,\nArizona, as mall matter of the second\nclass.\nSUBSCRIPTION RATES.\nBy mail, dally, one year WOO\nWeekly, one year 2.00\nCash in advance.\nBY CARRIER.\nDally, per month 73 cts\nUNlONfcQLrL\nPHOENIX. ARIZONA, JL\'XLO U. 1906.\nA BEGGING SENATORSHIP.\nThe rejection by that unambitious ag\nlku\'turist, the Hon. Dwtght Coburn. of\nthe appointment to the vacant Kansas\nsenatorship need not be attributed to\nhis utter disregard of pomp and pow\ner. Mr. Coburn is already in power,\nvast power, practically illimitab\'.e pow\ner. He is lord of the immeasurable\ncornfie\'d

In [8]:
requests.get(page_url).json()

{'issue': {'date_issued': '1906-06-10',
  'url': 'http://chroniclingamerica.loc.gov/lccn/sn84020558/1906-06-10/ed-1.json'},
 'jp2': 'http://chroniclingamerica.loc.gov/lccn/sn84020558/1906-06-10/ed-1/seq-2.jp2',
 'ocr': 'http://chroniclingamerica.loc.gov/lccn/sn84020558/1906-06-10/ed-1/seq-2/ocr.xml',
 'pdf': 'http://chroniclingamerica.loc.gov/lccn/sn84020558/1906-06-10/ed-1/seq-2.pdf',
 'sequence': 2,
 'text': 'http://chroniclingamerica.loc.gov/lccn/sn84020558/1906-06-10/ed-1/seq-2/ocr.txt',
 'title': {'name': 'Arizona republican.',
  'url': 'http://chroniclingamerica.loc.gov/lccn/sn84020558.json'}}

Now we're going to change up a little bit and do some search terms. This is a bit different, because now the search is based on a relevancy ranking (unlike the previous one). So click on this search to see what it looks like:

http://chroniclingamerica.loc.gov/search/pages/results/?proxtext=bedbugs

And then add "&format=json" to the end of it to get the API version again:

In [11]:
b = requests.get('http://chroniclingamerica.loc.gov/search/pages/results/?proxtext=bedbugs&format=json').json()

We can do the same thing we did before, and just see how many results there are:

In [12]:
b['totalItems']

29984

And we can look at the items themselves again:

In [13]:
b['items'][0]['url']

'http://chroniclingamerica.loc.gov/lccn/sn83045487/1912-06-18/ed-1/seq-31.json'

Maybe now, we want to try something more complicated, like asking, "which state has the most problem with bedbugs" as is evidenced by the entirely scientific first hundred results for the search "bedbugs" on ChronAm. Note that we added "&rows=100" to this, which means "give me the first hundred results". Most search engines (including this one) let us paginate the results and pick which page we want. Without picking one, we get the first one. So, the top 100 most relevant results.

In [14]:
p = requests.get("http://chroniclingamerica.loc.gov/search/pages/results/?proxtext=bedbugs&rows=100&format=json").json()

Now we're going to loop over all the results:

In [16]:
for page in p['items']:
    print(page['state'])

['Illinois']
['Illinois']
['Illinois']
['Illinois']
['Illinois']
['Illinois']
['Illinois']
['Illinois']
['South Dakota']
['Illinois']
['Oregon']
['Illinois']
['Illinois']
['Mississippi']
['Missouri']
['Kansas']
['Minnesota']
['South Dakota']
['Oregon']
['Pennsylvania']
['Arizona']
['Indiana']
['Iowa']
['Connecticut']
['Indiana']
['Indiana']
['Missouri']
['Illinois']
['Illinois']
['Illinois']
['Illinois']
['Illinois']
['Illinois']
['Illinois']
['Illinois']
['Illinois']
['Kentucky']
['Illinois']
['Kansas']
['Illinois']
['Illinois']
['Kentucky']
['Texas']
['Kentucky']
['Ohio']
['Indiana']
['District of Columbia']
['Kentucky']
['Pennsylvania']
['Tennessee']
['Kentucky']
['Virginia']
['Indiana']
['Kentucky']
['Kentucky']
['Oregon']
['Indiana']
['Connecticut']
['Connecticut']
['Connecticut']
['Kentucky']
['Kentucky']
['Connecticut']
['Oklahoma']
['North Carolina']
['West Virginia']
['Montana']
['Tennessee']
['Arizona']
['Oregon']
['Oregon']
['Tennessee']
['Connecticut']
['Connecticut']
['Con

In [17]:
from collections import Counter

In [32]:
c = Counter()
for page in p['items']:
    c[page['state'][0]] += 1
    
c.most_common()

[('Illinois', 24),
 ('Connecticut', 10),
 ('Kentucky', 10),
 ('Indiana', 7),
 ('Oregon', 6),
 ('Mississippi', 4),
 ('Washington', 3),
 ('South Carolina', 3),
 ('Kansas', 3),
 ('Iowa', 3),
 ('Arizona', 3),
 ('District of Columbia', 3),
 ('Tennessee', 3),
 ('Pennsylvania', 2),
 ('South Dakota', 2),
 ('Missouri', 2),
 ('Oklahoma', 2),
 ('West Virginia', 1),
 ('Florida', 1),
 ('Montana', 1),
 ('Virginia', 1),
 ('Texas', 1),
 ('Minnesota', 1),
 ('Utah', 1),
 ('Ohio', 1),
 ('North Carolina', 1),
 ('Louisiana', 1)]

Now, it's easy enough to hate on Illinios about their obvious bedbug problem, but this is where understanding the results probably requires a human who understands the collections a little bit. Any guesses?